In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
rates = pd.read_csv("datasets/rates.csv")
rates

# Rate is in cents per miles (randomly generated)

,Load,Origin,Destination,Rate
0,1,Chicago,Denver,56
1,2,Chicago,Denver,31
2,3,Chicago,Denver,25
3,4,Chicago,Denver,55
4,5,Chicago,Denver,52
5,6,Chicago,Austin,28
6,7,Chicago,Austin,48
7,8,Chicago,Austin,49
8,9,Austin,Seattle,70
9,10,Austin,Portland,72


In [3]:
times = pd.read_csv("datasets/times.csv")
times

# time is in hours (from google maps)

,Origin,Destination,Time
0,Austin,Seattle,33
1,Austin,Portland,32
2,Austin,Boston,30
3,Austin,San Antonio,2
4,Austin,Chicago,18
5,Boston,Portland,46
6,Chicago,Denver,15
7,Chicago,Austin,18
8,Chicago,Seattle,30
9,Denver,Los Angeles,16


In [4]:
df = rates.merge(times, left_on = ["Origin", "Destination"], right_on = ["Origin", "Destination"], how = "left")
df

,Load,Origin,Destination,Rate,Time
0,1,Chicago,Denver,56,15
1,2,Chicago,Denver,31,15
2,3,Chicago,Denver,25,15
3,4,Chicago,Denver,55,15
4,5,Chicago,Denver,52,15
5,6,Chicago,Austin,28,18
6,7,Chicago,Austin,48,18
7,8,Chicago,Austin,49,18
8,9,Austin,Seattle,70,33
9,10,Austin,Portland,72,32


In [5]:
# start in Chicago

origin = "Chicago"

node1 = df[df["Origin"] == origin]
node1

,Load,Origin,Destination,Rate,Time
0,1,Chicago,Denver,56,15
1,2,Chicago,Denver,31,15
2,3,Chicago,Denver,25,15
3,4,Chicago,Denver,55,15
4,5,Chicago,Denver,52,15
5,6,Chicago,Austin,28,18
6,7,Chicago,Austin,48,18
7,8,Chicago,Austin,49,18
21,22,Chicago,Seattle,66,30


In [6]:
# node1 destinations

node2options = set(node1["Destination"])
node2options

{'Austin', 'Denver', 'Seattle'}

In [7]:
# focus on austin first

toAustin1 = [28, 18] # [rate, time]
toAustin2 = [48, 18]
toAustin3 = [49, 18]

# same miles because they're all going to the same city
# but different rates. 
# the best one is toAustin3 with 49 cents/mile

In [8]:
# focus on denver now

toDenver1 = [56, 15]
toDenver2 = [31, 15]
toDenver3 = [25, 15]
toDenver4 = [55, 15]
toDenver5 = [52, 15]

# pick toDenver4 because it's the highest rate of 55 c/m

In [9]:
# focus on seattle now

toSeattle1 = [66, 30]

In [10]:
# should i go with austin, denver, or seattle?
# going to seattle yields the highest rate.
# let's assume I have a capacity limit of 50 miles.

selected = [66, 30]

# the only load leaving seattle is portland [75, 4]
nextLoad = [75, 4]

In [11]:
rate_time = [selected[0] + nextLoad[0], selected[1] + nextLoad[1]]
rate_time

[141, 34]

In [12]:
# I have still have miles to drive. 
# the only available load is going from portland to denver [70, 20]
selected = [70, 20]
rate_time = [selected[0] + rate_time[0], selected[1] + rate_time[1]]
rate_time

[211, 54]

In [13]:
# denver goes to 3 spots 
# but i can't go anywhere because it
# would put me over the cap of 50 miles. 
# my tour ends here. 

df[df["Origin"] == "Denver"]

,Load,Origin,Destination,Rate,Time
11,12,Denver,Los Angeles,52,16
12,13,Denver,San Francisco,40,20
17,18,Denver,Chicago,50,15


In [14]:
# how can i automate this now?

In [15]:
def calculateRateTime(nextNode, dt=None):
    if (dt == None):
        dt = [0, 0]
        
    dt = [dt[0] + nextNode[0], dt[1] + nextNode[1]]
    return(dt)  

In [16]:
calculateRateTime([5,3], [1,1])

[6, 4]

In [17]:
def getRateTime(record):
    rate = record["Rate"]
    time = record["Time"]
    return([rate, time])

In [47]:
for i in range(0, 5):
    row = df.iloc[i, ]
    ratetime = getRateTime(row)
    print(ratetime)
    print("-----")

[56, 15]
-----
[31, 15]
-----
[25, 15]
-----
[55, 15]
-----
[52, 15]
-----


In [19]:
class Tour:
    def __init__(self):
        self.loads = []
        self.metrics = [0, 0]
        
    def append(self, val):
        self.loads.append(val)
        return self.loads

In [20]:
def findDestinations(node):
    return(list(set(df[df["Origin"] == node]["Destination"])))

In [21]:
findDestinations(origin)

['Austin', 'Denver', 'Seattle']

In [22]:
origin = "Austin" #origin

In [71]:
def initializeTour(origin):
    tourList = []
    for nextDest in findDestinations(origin): 
        destinations = df[(df["Origin"] == origin) & (df["Destination"] == nextDest)]

        for r in range(0, len(destinations)):
            loadID = destinations.iloc[r, ]["Load"]        
            rt = getRateTime(destinations.iloc[r, ])
            tour = Tour()
            tour.append(loadID)
            tour.metrics = calculateRateTime(tour.metrics, rt)
            tourList.append(tour)
    return tourList

In [72]:
# build on next tour

def buildTour(tourList):
    for i in tourList:
        lastLoadID = i.loads[-1]
        lastLoadCity = df[df["Load"] == lastLoadID].iloc[0]["Destination"]
        nextCities = findDestinations(lastLoadCity)

        for c in nextCities:
            loadsToDestinations = df[(df["Origin"] == lastLoadCity) & (df["Destination"] == c)]

            for l in range(0, len(loadsToDestinations)):
                load = loadsToDestinations.iloc[l]
                if (load["Load"] not in i.loads):
                    rt = getRateTime(load)
                    tour = copy.deepcopy(i)
                    tour.append(load["Load"])
                    tour.metrics = calculateRateTime(tour.metrics, rt)
                    if (tour.metrics[1] <= 100):
                        tourList.append(tour)    
    return tourList


In [77]:
tourList = initializeTour("Portland")

for tour in tourList:
    print("Load Series", tour.loads)
    print("Tour Metrics", tour.metrics)
    print("----------------------")

Load Series [19]
Tour Metrics [70, 20]
----------------------


In [78]:
tourList = buildTour(tourList)
len(tourList)

120

In [79]:
for t in tourList:
    print("Tour:", t.loads)
    print("Metrics:", t.metrics)
    print("=============")

Tour: [19]
Metrics: [70, 20]
Tour: [19, 12]
Metrics: [122, 36]
Tour: [19, 18]
Metrics: [120, 35]
Tour: [19, 13]
Metrics: [110, 40]
Tour: [19, 18, 6]
Metrics: [148, 53]
Tour: [19, 18, 7]
Metrics: [168, 53]
Tour: [19, 18, 8]
Metrics: [169, 53]
Tour: [19, 18, 1]
Metrics: [176, 50]
Tour: [19, 18, 2]
Metrics: [151, 50]
Tour: [19, 18, 3]
Metrics: [145, 50]
Tour: [19, 18, 4]
Metrics: [175, 50]
Tour: [19, 18, 5]
Metrics: [172, 50]
Tour: [19, 18, 22]
Metrics: [186, 65]
Tour: [19, 13, 17]
Metrics: [190, 52]
Tour: [19, 18, 6, 15]
Metrics: [218, 55]
Tour: [19, 18, 6, 11]
Metrics: [195, 83]
Tour: [19, 18, 6, 10]
Metrics: [220, 85]
Tour: [19, 18, 6, 24]
Metrics: [189, 85]
Tour: [19, 18, 6, 25]
Metrics: [198, 71]
Tour: [19, 18, 6, 9]
Metrics: [218, 86]
Tour: [19, 18, 7, 15]
Metrics: [238, 55]
Tour: [19, 18, 7, 11]
Metrics: [215, 83]
Tour: [19, 18, 7, 10]
Metrics: [240, 85]
Tour: [19, 18, 7, 24]
Metrics: [209, 85]
Tour: [19, 18, 7, 25]
Metrics: [218, 71]
Tour: [19, 18, 7, 9]
Metrics: [238, 86]
Tour: [